## Part 1d: TensorFlow - Monte Carlo Dropout

**Description:**

This Colab demonstrates the Monte Carlo Dropout technique in TensorFlow Keras. Unlike standard evaluation where dropout layers are turned off, Monte Carlo Dropout involves keeping the dropout layers active during inference. By performing multiple forward passes with dropout enabled and averaging the predictions, we can obtain not only a prediction but also an estimate of the model's uncertainty.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the digits dataset
digits = load_digits()
X, y = digits.data, digits.target

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model with Dropout
mc_dropout_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

mc_dropout_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mc_dropout_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=0)

# Make multiple predictions with dropout enabled
num_samples = 100
predictions = np.stack([mc_dropout_model(X_test, training=True) for _ in range(num_samples)])
mean_prediction = np.mean(predictions, axis=0)

# Evaluate the performance based on the average prediction
predicted_classes = np.argmax(mean_prediction, axis=1)
accuracy = np.mean(predicted_classes == y_test)

print("\nMonte Carlo Dropout - Test Accuracy (averaged predictions):", accuracy)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Monte Carlo Dropout - Test Accuracy (averaged predictions): 0.9833333333333333


## Results for Part 1d: TensorFlow - Monte Carlo Dropout

In this experiment, we explored the Monte Carlo Dropout technique using a simple neural network with a Dropout layer (dropout rate of 0.5) trained on the `digits` dataset. During the evaluation phase, instead of turning off the Dropout layer as is standard, we kept it active (`training=True`) and performed multiple forward passes (100 times) for each test sample. The final prediction for each sample was obtained by averaging the predictions from these multiple passes.

The test accuracy achieved using the averaged predictions from Monte Carlo Dropout was:

* **Monte Carlo Dropout - Test Accuracy (averaged predictions):** 0.9833

**Analysis:**

The result indicates that using Monte Carlo Dropout during inference yielded a high test accuracy of 0.9833 on the `digits` dataset.

Monte Carlo Dropout serves two primary purposes:

1.  **Improved Prediction Robustness:** By averaging multiple predictions with different sets of neurons being randomly dropped out, the final prediction is often more robust and less susceptible to the peculiarities of individual network configurations.
2.  **Uncertainty Estimation:** The variance or entropy of the multiple predictions can provide an estimate of the model's uncertainty in its classification. Higher variance or entropy suggests lower confidence in the prediction. While we didn't explicitly calculate uncertainty in this Colab, the process of generating multiple predictions lays the groundwork for doing so.

The high accuracy achieved with Monte Carlo Dropout suggests that for this task and model, keeping dropout active during inference can lead to good generalization performance. The slight difference compared to the standard evaluation (which we didn't explicitly run in this Colab for the dropout model but was around 0.9778 in Part 1b) highlights the potential benefits of this technique.